<span style="color:black"><span style = "font-size:30px"> VAE model training</span>
&nbsp;&nbsp;&nbsp;
   
    
In this process, we built the variational autoencoder (VAE) model to generate new promoters.

1.	Import python modules required for training a VAE model and load the training dataset generated in the previous section (Acquisition of promoter dataset).

In [1]:
# 1.

from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
from keras.layers.convolutional import MaxPooling2D
from sklearn.metrics import mean_squared_error
import theano
from keras import optimizers
from tensorflow.keras import datasets, layers, models
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_probability as tfp
import time
data = pd.read_excel('/NAS/home/yychoi/research/capstone1/bio-ai-capstone/Training dataset.xlsx', engine = 'openpyxl')

Using TensorFlow backend.


In [2]:
import os
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(device_lib.list_local_devices() )

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15643558151910101587
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8241022691747173243
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
data.head()

,Promoter,Reads
0,CACCTCCAAATTATCTAAGTTAGCCAAATACCAAGAAGATTGGGCA...,106
1,TTTTGGGGAAACCGGCCGGGTCAGATTTAATCAGCGGCAACATCAC...,643
2,CCAAACGATGCTGAAGTTTACCGTTGCTGGTGAGCAGCAATAGTCC...,372
3,CAGCAACTCTTAACGGGAAATCCCAATGGTCCCTGGCAGAAAAAAT...,834
4,CCTGAATATCTCCAGGGTTATACCGCCCCCGATGAAGCTTTTGTTT...,1496


2.	Define the 'One-hot encoding' (OHE) function. OHE is a method that converts categorical variables to numerical ones in an interpretable format. Through the OHE, we can vectorize the DNA sequences, i.e., A : [1,0,0,0], C : [0,1,0,0], G : [0,0,1,0], T : [0,0,0,1]. The 'one-hot encoding' function converts 100-base-pair promoter sequences to 1x1x4x100-sized tensors. To avoid information loss at both ends of a promoter during model training, we put the 1x1x4x10-sized zero tensor at each end to yield a 1x1x4x120-sized tensor for each promoter. Through the OHE for all 3712 promoters, you can obtain a 3712x1x4x120-sized tensor.

In [4]:
# 2.

def one_hot_encoding(df, seq_column, expression):
    bases = ['A','C','G','T']
    base_dict = dict(zip(bases,range(4)))
    n = len(df)
    total_width = df[seq_column].str.len().max()+20
    X = np.zeros((n,1,4,total_width))
    seqs = df[seq_column].values
    for i in range(n):
        seq = seqs[i]
        for b in range(len(seq)):
            X[i,0,base_dict[seq[b]], b+10+100-len(seq)] = 1    
    X = X.astype(theano.config.floatX)
    return X, total_width

X, total_width = one_hot_encoding(data,'Promoter','Reads')

In [5]:
print(X[1])
print(X.shape)

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0.
   0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
   0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0.
   1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
   0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
   0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
   0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0.
   0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.
   1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
   1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
   0. 0. 0. 1. 0. 0. 0. 0.

3.	Define the class object of convolutional variational autoencoder (CVAE). The CVAE model contains ‘an encoder’ and ‘decoder’. The ‘encoder’ encodes the promoter tensor (1X1X4X120) into latent space. The ‘decoder’ converts a point of latent space into the generated data (1X1X4X120) showing genuine training dataset features. The size of the latent space can be specified by assigning a size number to the ‘latent_dim’ variable. The encoded inputs are converted into ‘mean’ and ‘logvar’ tensors. 

In [6]:
# 3.
tf.keras.backend.set_floatx('float32')
class VAE(tf.keras.Model):

    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(1,4,120)),
                tf.keras.layers.Conv2D(
                    filters=16, kernel_size=(4,35), strides=(1, 1), activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Conv2D(
                    filters=16, kernel_size=(1,21), strides=(1, 1), activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Conv2D(
                    filters=16, kernel_size=(1,15), strides=(1, 1), activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(latent_dim + latent_dim),
            ]
        )
        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=1*4*120, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(1, 4, 120)),
                tf.keras.layers.Conv2DTranspose(
                    filters=16, kernel_size=(1,15), strides=(1,1), padding='same',
                    activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Conv2DTranspose(
                    filters=16, kernel_size=(1,21), strides=(1,1), padding='same',
                    activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Conv2DTranspose(
                    filters=16, kernel_size=(1,35), strides=(1,1), padding='same',
                    activation='relu',data_format = 'channels_first'),
                tf.keras.layers.Conv2DTranspose(
                    filters=1, kernel_size=(4,29), strides=1, padding='same', data_format = 'channels_first'),
            ]
        )

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

4.	Define the loss function and train steps. The 'log_normal_pdf' function is used to calculate the probability density function, and the 'compute_loss' function is used to calculate the Kullback-Leibler divergence (KLD). 

In [7]:
# 4.

optimizer = tf.keras.optimizers.Adam(1e-4)
def log_normal_pdf(sample, mean, logvar, raxis=1):
    
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)

def compute_loss(model, x):
    
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = tf.dtypes.cast(model.decode(z),tf.float64)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = tf.dtypes.cast(log_normal_pdf(z, 0., 0.),tf.float64)
    logqz_x = tf.dtypes.cast(log_normal_pdf(z, mean, logvar),tf.float64)
    
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

5.	Set up a model. You can set the number of epochs (how many times the training function runs) and the dimension of the latent space. You can also decide the number of synthetic promoters to be generated by assigning the desired number to the variable 'num_examples_to_generate'.

In [8]:
# 5.

epochs = 1000
latent_dim = 2
num_examples_to_generate = 10000
random_vector_for_generation = tf.random.normal(
    shape=[num_examples_to_generate, latent_dim])
model = VAE(latent_dim)

6.	Model training procedure

In [9]:
# 6.

X = np.expand_dims(X,axis = 0)

#elbolist = []

for epoch in range(1, epochs + 1):
    start_time = time.time()
    for tot_X in X:
        train_step(model, tot_X, optimizer)
    end_time = time.time()

    loss = tf.keras.metrics.Mean()
    for tot_X in X:
        loss(compute_loss(model, tot_X))
    elbo = -loss.result()
    print(elbo)
    #elbolist.append(elbo)
    
    
def decoder2seq(x):
    seq = ''
    for i in range(np.shape(x)[-1]-20):
        Ascalar = x[0][0][i+10].numpy()
        Cscalar = x[0][1][i+10].numpy()
        Gscalar = x[0][2][i+10].numpy()
        Tscalar = x[0][3][i+10].numpy()
        maxcalar = max(Ascalar, Cscalar, Gscalar, Tscalar)
        if Ascalar == maxcalar:
            seq = seq+'A'
        elif Cscalar == maxcalar:
            seq = seq+'C'
        elif Gscalar == maxcalar:
            seq = seq+'G'
        else:
            seq = seq+'T'
    return seq


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(-332.33603, shape=(), dtype=float32)
tf.Tensor(-332.14807, shape=(), dtype=float32)
tf.Tensor(-331.95428, shape=(), dtype=float32)
tf.Tensor(-331.7248, shape=(), dtype=float32)
tf.Tensor(-331.5087, shape=(), dtype=float32)
tf.Tensor(-331.24872, shape=(), dtype=float32)
tf.Tensor(-330.99075, shape=(), dtype=float32)
tf.Tensor(-330.7105, shape=(), dtype=float32)
tf.Tensor(-330.37433, shape=(), dtype=float32)
tf.Tensor(-330.05246, shape=(), dtype=float32)
tf.Tensor(-329.70325, shape=(), dtype=flo

7. Save the VAE model and synthetized promoter dataset. You can save the VAE model separately by the encoder ('cyano_encoder.h5') and decoder ('cyano_decoder.h5').

In [ ]:
model.encoder.save('cyano_encode.h5')
model.decoder.save('cyano_decode.h5')

In [1]:
# 7.
'''
model.encoder.save('cyano_encode.h5')
model.decoder.save('cyano_decode.h5')
'''
aplist = []
for i in range(10000):
    aplist.append(decoder2seq(model.decode(random_vector_for_generation)[i]))
    
random_vector_for_generation = tf.random.normal(shape=[10000, 2])
ddff = pd.DataFrame(aplist)
ddff.columns = ['Generated promoters']
ddff.to_excel('Generated promoter sequences 100bp.xlsx', index= False)

NameError: name 'decoder2seq' is not defined